<a href="https://colab.research.google.com/github/byhqsr/CodeRepo/blob/main/Feature_Engineering_ICoDSA_2021_Louis_Owen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Author: [Louis Owen](https://louisowen6.github.io/)

Download this [data](https://drive.google.com/file/d/17gHvgAEcpy7NjUHRU9Ott18pYdCeLveQ/view) and try to perform the exact feature engineering in the previous study case. 

Note: no need to worry about `is_cross_sell` column in this dataset.

# Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/Public Speaker/ICoDSA 2021

/content/gdrive/MyDrive/Public Speaker/ICoDSA 2021


# Import Package

In [ ]:
import pandas as pd
import numpy as np

# Import Data

In [ ]:
df = pd.read_csv('test.csv')

# Initial EDA

In [ ]:
df.sample(5)

,account_id,order_id,member_duration_days,gender,trip,service_class,price,is_tx_promo,no_of_seats,airlines_name,route,visited_city,log_transaction
4465,ec224f599d4a1a97dbd0a577ef5f2223,37b1a2e6dbf722ce254cd46aaa14bbb6,707,F,roundtrip,ECONOMY,4065126.00,NO,2,0a102015e48c1f68e121acc99fca9a05,CGK - DPS,"'['Medan', 'Bali', 'Jakarta']'","'[8650708.83, 2476530.38, 5745383.85, 4065126.0]'"
6827,8fde94e9aab1312a33ee0ccf8b08e16a,2fbb5c068a11cc4524c19b3cc52e9f37,2357,M,trip,ECONOMY,2060115.07,YES,1,ad5bef60d81ea077018f4d50b813153a,CGK - DPS,"'['Jakarta', 'Medan', 'Bali']'","'[1082500.74, 4673968.76, 2146519.7, 2060115.07]'"
1578,db792894853c7094dd28353d2e09c68f,c0a88a07992425a8d1020fcc413a12bc,438,M,trip,ECONOMY,4148612.00,YES,4,6c483c0812c96f8ec43bb0ff76eaf716,CGK - DPS,"'['Medan', 'Bali', 'Jakarta']'","'[6941346.35, 4148612.0]'"
653,fa1f8da3294f672fd3acc0dc15b02a65,48d75c1e4c9567fc948f54c11f0bb001,1455,F,roundtrip,ECONOMY,2247700.00,YES,1,33199710eb822fbcfd0dc793f4788d30,CGK - DPS,"'['Medan', 'Bali', 'Jakarta']'","'[9814253.54, 3263390.0, 3263390.0, 1631695.0,..."
5344,edf1b92660089f63bf316d7c0525f08c,8c5194579681fbee25a7449aace1082d,338,M,round,ECONOMY,1465230.00,NO,1,6c483c0812c96f8ec43bb0ff76eaf716,CGK - DPS,"'['Jakarta', 'Bali', 'Medan', 'Jogjakarta', 'S...","'[9117984.91, 2470066.57, 1465230.0]'"


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   account_id            10000 non-null  object 
 1   order_id              10000 non-null  object 
 2   member_duration_days  10000 non-null  int64  
 3   gender                10000 non-null  object 
 4   trip                  10000 non-null  object 
 5   service_class         10000 non-null  object 
 6   price                 10000 non-null  float64
 7   is_tx_promo           10000 non-null  object 
 8   no_of_seats           10000 non-null  int64  
 9   airlines_name         10000 non-null  object 
 10  route                 10000 non-null  object 
 11  visited_city          10000 non-null  object 
 12  log_transaction       10000 non-null  object 
dtypes: float64(1), int64(2), object(10)
memory usage: 1015.8+ KB


# Feature Engineering

## Construct Aggregation Statistics Features from Log Transaction

In [ ]:
from ast import literal_eval

def extract_log_trx_stats(log_series):
    # Dictionary to save the statistics value for each rows
    log_trx_stats_dict = {'min_trx':[],'max_trx':[],
                          'avg_trx':[],'std_trx':[],'cnt_trx':[]}

    for i in range(len(log_series)): # Lopping through each rows
        # Convert from string of list to list
        trx_list = literal_eval(literal_eval(log_series.iloc[i]))

        # Extract Aggreation Statistics
        log_trx_stats_dict['min_trx'].append(np.min(trx_list))
        log_trx_stats_dict['max_trx'].append(np.max(trx_list))
        log_trx_stats_dict['avg_trx'].append(np.mean(trx_list))
        log_trx_stats_dict['std_trx'].append(np.std(trx_list))
        log_trx_stats_dict['cnt_trx'].append(len(trx_list))

    return pd.DataFrame(log_trx_stats_dict)

In [ ]:
df = pd.concat([df,extract_log_trx_stats(df['log_transaction'])],axis=1)

In [ ]:
df

,account_id,order_id,member_duration_days,gender,trip,service_class,price,is_tx_promo,no_of_seats,airlines_name,route,visited_city,log_transaction,min_trx,max_trx,avg_trx,std_trx,cnt_trx
0,89a5fadd4d596610ff56044b9a0b1f4f,5ca64fd80a069208e3c0aa05dd580fb8,1754,M,trip,ECONOMY,3143400.00,YES,3,e35de6a36d385711a660c72c0286154a,CGK - DPS,"'['Bali', 'Jakarta', 'Medan']'","'[568508.04, 245284.05, 748834.2, 7930226.7, 6...",245284.05,7930226.70,3.151466e+06,2.985184e+06,6
1,86b28323bec6d938d47cee887e509b28,aca60904549a8a5958fe7a642efcb534,1084,F,trip,ECONOMY,2166607.04,NO,2,e35de6a36d385711a660c72c0286154a,CGK - DPS,"'['Medan', 'Bali', 'Jakarta']'","'[9916349.28, 7404847.79, 4697159.89, 2166607....",2166607.04,9916349.28,6.046241e+06,2.902375e+06,4
2,36ef956ac3ef963c48e67327a4b6cc78,1771011e3adec5db9f30d15b3d439711,2378,M,round,ECONOMY,1239615.00,NO,1,ad5bef60d81ea077018f4d50b813153a,CGK - DPS,"'['Jakarta', 'Medan', 'Bali']'","'[249468.18, 1854801.11, 3311742.08, 4002723.4...",249468.18,7757790.96,3.069357e+06,2.438791e+06,6
3,f7821289404d44db50eb2edd4f82ea5b,6fc1b7d590c2a8c539ce56397403194d,576,F,trip,ECONOMY,1984060.00,YES,2,33199710eb822fbcfd0dc793f4788d30,CGK - DPS,"'['Jakarta', 'Bali', 'Medan', 'Jogjakarta', 'S...","'[6137034.61, 58939.21, 1984060.0]'",58939.21,6137034.61,2.726678e+06,2.536326e+06,3
4,f62f33d1de5aabc919b69b1b5697f27a,c1f4712f60cd758e773555690d148764,862,F,trip,ECONOMY,2990624.00,YES,1,74c5549aa99d55280a896ea50068a211,CGK - DPS,"'['Bali', 'Jakarta', 'Medan']'","'[9653724.83, 6417438.6, 2990624.0]'",2990624.00,9653724.83,6.353929e+06,2.720570e+06,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,998d43683c1ba7a49e1ac8392b738271,a3a06ed271bd87f0e987be26f8f200f8,509,M,trip,ECONOMY,5206890.00,NO,3,e35de6a36d385711a660c72c0286154a,CGK - DPS,"'['Medan', 'Bali', 'Jakarta']'","'[7012695.13, 5367357.5, 5324636.51, 5368200.3...",5206890.00,7012695.13,5.655956e+06,6.809209e+05,5
9996,baead48bd98db56344745902906c1f08,2d4ba920b453a22e079c0888acea9bba,2146,M,trip,ECONOMY,3339440.00,NO,2,74c5549aa99d55280a896ea50068a211,CGK - DPS,"'['Jakarta', 'Medan', 'Bali']'","'[9246992.23, 7841884.74, 5223823.74, 6572676....",1926938.00,9246992.23,5.108861e+06,2.241444e+06,9
9997,6b280411d8c49c5572e0499eaebdac18,f22e17eebf2b7305906161b517391aef,1732,M,trip,ECONOMY,1055168.40,YES,1,0a102015e48c1f68e121acc99fca9a05,CGK - DPS,"'['Jakarta', 'Bali', 'Medan', 'Jogjakarta', 'S...","'[6049561.3, 3455632.61, 5524000.64, 995410.0,...",870857.00,6049561.30,2.991772e+06,2.168394e+06,6
9998,716c79a0238b8aa7da0089d6ba2c70b2,f981b0ea71a2674216078c22f6d8ec52,734,F,trip,ECONOMY,4049240.00,NO,4,ad5bef60d81ea077018f4d50b813153a,CGK - DPS,"'['Bali', 'Jakarta', 'Medan']'","'[3396321.28, 7790742.28, 4049240.0]'",3396321.28,7790742.28,5.078768e+06,1.936092e+06,3


## Construct Is Visited City Features

In [ ]:
df['visited_jogja'] = df['visited_city'].apply(lambda x: 1 if 'Jogjakarta' in x else 0)
df['visited_semarang'] = df['visited_city'].apply(lambda x: 1 if 'Semarang' in x else 0)
df['visited_surabaya'] = df['visited_city'].apply(lambda x: 1 if 'Surabaya' in x else 0)
df['visited_aceh'] = df['visited_city'].apply(lambda x: 1 if 'Aceh' in x else 0)
df['visited_manado'] = df['visited_city'].apply(lambda x: 1 if 'Manado' in x else 0)

In [ ]:
df

,account_id,order_id,member_duration_days,gender,trip,service_class,price,is_tx_promo,no_of_seats,airlines_name,route,visited_city,log_transaction,min_trx,max_trx,avg_trx,std_trx,cnt_trx,visited_jogja,visited_semarang,visited_surabaya,visited_aceh,visited_manado
0,89a5fadd4d596610ff56044b9a0b1f4f,5ca64fd80a069208e3c0aa05dd580fb8,1754,M,trip,ECONOMY,3143400.00,YES,3,e35de6a36d385711a660c72c0286154a,CGK - DPS,"'['Bali', 'Jakarta', 'Medan']'","'[568508.04, 245284.05, 748834.2, 7930226.7, 6...",245284.05,7930226.70,3.151466e+06,2.985184e+06,6,0,0,0,0,0
1,86b28323bec6d938d47cee887e509b28,aca60904549a8a5958fe7a642efcb534,1084,F,trip,ECONOMY,2166607.04,NO,2,e35de6a36d385711a660c72c0286154a,CGK - DPS,"'['Medan', 'Bali', 'Jakarta']'","'[9916349.28, 7404847.79, 4697159.89, 2166607....",2166607.04,9916349.28,6.046241e+06,2.902375e+06,4,0,0,0,0,0
2,36ef956ac3ef963c48e67327a4b6cc78,1771011e3adec5db9f30d15b3d439711,2378,M,round,ECONOMY,1239615.00,NO,1,ad5bef60d81ea077018f4d50b813153a,CGK - DPS,"'['Jakarta', 'Medan', 'Bali']'","'[249468.18, 1854801.11, 3311742.08, 4002723.4...",249468.18,7757790.96,3.069357e+06,2.438791e+06,6,0,0,0,0,0
3,f7821289404d44db50eb2edd4f82ea5b,6fc1b7d590c2a8c539ce56397403194d,576,F,trip,ECONOMY,1984060.00,YES,2,33199710eb822fbcfd0dc793f4788d30,CGK - DPS,"'['Jakarta', 'Bali', 'Medan', 'Jogjakarta', 'S...","'[6137034.61, 58939.21, 1984060.0]'",58939.21,6137034.61,2.726678e+06,2.536326e+06,3,1,1,0,0,0
4,f62f33d1de5aabc919b69b1b5697f27a,c1f4712f60cd758e773555690d148764,862,F,trip,ECONOMY,2990624.00,YES,1,74c5549aa99d55280a896ea50068a211,CGK - DPS,"'['Bali', 'Jakarta', 'Medan']'","'[9653724.83, 6417438.6, 2990624.0]'",2990624.00,9653724.83,6.353929e+06,2.720570e+06,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,998d43683c1ba7a49e1ac8392b738271,a3a06ed271bd87f0e987be26f8f200f8,509,M,trip,ECONOMY,5206890.00,NO,3,e35de6a36d385711a660c72c0286154a,CGK - DPS,"'['Medan', 'Bali', 'Jakarta']'","'[7012695.13, 5367357.5, 5324636.51, 5368200.3...",5206890.00,7012695.13,5.655956e+06,6.809209e+05,5,0,0,0,0,0
9996,baead48bd98db56344745902906c1f08,2d4ba920b453a22e079c0888acea9bba,2146,M,trip,ECONOMY,3339440.00,NO,2,74c5549aa99d55280a896ea50068a211,CGK - DPS,"'['Jakarta', 'Medan', 'Bali']'","'[9246992.23, 7841884.74, 5223823.74, 6572676....",1926938.00,9246992.23,5.108861e+06,2.241444e+06,9,0,0,0,0,0
9997,6b280411d8c49c5572e0499eaebdac18,f22e17eebf2b7305906161b517391aef,1732,M,trip,ECONOMY,1055168.40,YES,1,0a102015e48c1f68e121acc99fca9a05,CGK - DPS,"'['Jakarta', 'Bali', 'Medan', 'Jogjakarta', 'S...","'[6049561.3, 3455632.61, 5524000.64, 995410.0,...",870857.00,6049561.30,2.991772e+06,2.168394e+06,6,1,1,0,0,0
9998,716c79a0238b8aa7da0089d6ba2c70b2,f981b0ea71a2674216078c22f6d8ec52,734,F,trip,ECONOMY,4049240.00,NO,4,ad5bef60d81ea077018f4d50b813153a,CGK - DPS,"'['Bali', 'Jakarta', 'Medan']'","'[3396321.28, 7790742.28, 4049240.0]'",3396321.28,7790742.28,5.078768e+06,1.936092e+06,3,0,0,0,0,0


## Encode Airlines Name

In [ ]:
df['airlines_name'].unique()

array(['e35de6a36d385711a660c72c0286154a',
       'ad5bef60d81ea077018f4d50b813153a',
       '33199710eb822fbcfd0dc793f4788d30',
       '74c5549aa99d55280a896ea50068a211',
       '0a102015e48c1f68e121acc99fca9a05',
       '6c483c0812c96f8ec43bb0ff76eaf716'], dtype=object)

In [ ]:
airlines_name_dict = {'e35de6a36d385711a660c72c0286154a':'A',
                      'ad5bef60d81ea077018f4d50b813153a': 'B',
                      '33199710eb822fbcfd0dc793f4788d30': 'C',
                      '74c5549aa99d55280a896ea50068a211': 'D',
                      '0a102015e48c1f68e121acc99fca9a05': 'E',
                      '6c483c0812c96f8ec43bb0ff76eaf716': 'F',
                      }

In [ ]:
df['airlines_name'] = df['airlines_name'].map(airlines_name_dict)

In [ ]:
df

,account_id,order_id,member_duration_days,gender,trip,service_class,price,is_tx_promo,no_of_seats,airlines_name,route,visited_city,log_transaction,min_trx,max_trx,avg_trx,std_trx,cnt_trx,visited_jogja,visited_semarang,visited_surabaya,visited_aceh,visited_manado
0,89a5fadd4d596610ff56044b9a0b1f4f,5ca64fd80a069208e3c0aa05dd580fb8,1754,M,trip,ECONOMY,3143400.00,YES,3,A,CGK - DPS,"'['Bali', 'Jakarta', 'Medan']'","'[568508.04, 245284.05, 748834.2, 7930226.7, 6...",245284.05,7930226.70,3.151466e+06,2.985184e+06,6,0,0,0,0,0
1,86b28323bec6d938d47cee887e509b28,aca60904549a8a5958fe7a642efcb534,1084,F,trip,ECONOMY,2166607.04,NO,2,A,CGK - DPS,"'['Medan', 'Bali', 'Jakarta']'","'[9916349.28, 7404847.79, 4697159.89, 2166607....",2166607.04,9916349.28,6.046241e+06,2.902375e+06,4,0,0,0,0,0
2,36ef956ac3ef963c48e67327a4b6cc78,1771011e3adec5db9f30d15b3d439711,2378,M,round,ECONOMY,1239615.00,NO,1,B,CGK - DPS,"'['Jakarta', 'Medan', 'Bali']'","'[249468.18, 1854801.11, 3311742.08, 4002723.4...",249468.18,7757790.96,3.069357e+06,2.438791e+06,6,0,0,0,0,0
3,f7821289404d44db50eb2edd4f82ea5b,6fc1b7d590c2a8c539ce56397403194d,576,F,trip,ECONOMY,1984060.00,YES,2,C,CGK - DPS,"'['Jakarta', 'Bali', 'Medan', 'Jogjakarta', 'S...","'[6137034.61, 58939.21, 1984060.0]'",58939.21,6137034.61,2.726678e+06,2.536326e+06,3,1,1,0,0,0
4,f62f33d1de5aabc919b69b1b5697f27a,c1f4712f60cd758e773555690d148764,862,F,trip,ECONOMY,2990624.00,YES,1,D,CGK - DPS,"'['Bali', 'Jakarta', 'Medan']'","'[9653724.83, 6417438.6, 2990624.0]'",2990624.00,9653724.83,6.353929e+06,2.720570e+06,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,998d43683c1ba7a49e1ac8392b738271,a3a06ed271bd87f0e987be26f8f200f8,509,M,trip,ECONOMY,5206890.00,NO,3,A,CGK - DPS,"'['Medan', 'Bali', 'Jakarta']'","'[7012695.13, 5367357.5, 5324636.51, 5368200.3...",5206890.00,7012695.13,5.655956e+06,6.809209e+05,5,0,0,0,0,0
9996,baead48bd98db56344745902906c1f08,2d4ba920b453a22e079c0888acea9bba,2146,M,trip,ECONOMY,3339440.00,NO,2,D,CGK - DPS,"'['Jakarta', 'Medan', 'Bali']'","'[9246992.23, 7841884.74, 5223823.74, 6572676....",1926938.00,9246992.23,5.108861e+06,2.241444e+06,9,0,0,0,0,0
9997,6b280411d8c49c5572e0499eaebdac18,f22e17eebf2b7305906161b517391aef,1732,M,trip,ECONOMY,1055168.40,YES,1,E,CGK - DPS,"'['Jakarta', 'Bali', 'Medan', 'Jogjakarta', 'S...","'[6049561.3, 3455632.61, 5524000.64, 995410.0,...",870857.00,6049561.30,2.991772e+06,2.168394e+06,6,1,1,0,0,0
9998,716c79a0238b8aa7da0089d6ba2c70b2,f981b0ea71a2674216078c22f6d8ec52,734,F,trip,ECONOMY,4049240.00,NO,4,B,CGK - DPS,"'['Bali', 'Jakarta', 'Medan']'","'[3396321.28, 7790742.28, 4049240.0]'",3396321.28,7790742.28,5.078768e+06,1.936092e+06,3,0,0,0,0,0


# Drop Columns

In [ ]:
df = df.drop(columns=['account_id','order_id','route','visited_city','log_transaction'])

# Final Version

In [ ]:
df

,member_duration_days,gender,trip,service_class,price,is_tx_promo,no_of_seats,airlines_name,min_trx,max_trx,avg_trx,std_trx,cnt_trx,visited_jogja,visited_semarang,visited_surabaya,visited_aceh,visited_manado
0,1754,M,trip,ECONOMY,3143400.00,YES,3,A,245284.05,7930226.70,3.151466e+06,2.985184e+06,6,0,0,0,0,0
1,1084,F,trip,ECONOMY,2166607.04,NO,2,A,2166607.04,9916349.28,6.046241e+06,2.902375e+06,4,0,0,0,0,0
2,2378,M,round,ECONOMY,1239615.00,NO,1,B,249468.18,7757790.96,3.069357e+06,2.438791e+06,6,0,0,0,0,0
3,576,F,trip,ECONOMY,1984060.00,YES,2,C,58939.21,6137034.61,2.726678e+06,2.536326e+06,3,1,1,0,0,0
4,862,F,trip,ECONOMY,2990624.00,YES,1,D,2990624.00,9653724.83,6.353929e+06,2.720570e+06,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,509,M,trip,ECONOMY,5206890.00,NO,3,A,5206890.00,7012695.13,5.655956e+06,6.809209e+05,5,0,0,0,0,0
9996,2146,M,trip,ECONOMY,3339440.00,NO,2,D,1926938.00,9246992.23,5.108861e+06,2.241444e+06,9,0,0,0,0,0
9997,1732,M,trip,ECONOMY,1055168.40,YES,1,E,870857.00,6049561.30,2.991772e+06,2.168394e+06,6,1,1,0,0,0
9998,734,F,trip,ECONOMY,4049240.00,NO,4,B,3396321.28,7790742.28,5.078768e+06,1.936092e+06,3,0,0,0,0,0
